## Lineair Regression Model 

We hebben gekozen om een Lineair Regression model eerst toe te passen.De reden hiervoor is dat onze target variabel continu waardes bevatten. Bovendien zijn bevatten de meeste features continu waardes of kunen *encoded* worden naar discrete waardes.

We zullen de volgende feature variabelen in onze Linear Regression model.
* **stm_oorz_groep** : *Aan de hand van deze feature, krijgen we en beter definitie van de bepaalde storing. Welke groep de oorzaak een bepaalde storing geven. *
* **stm_oor_code**   : *Zelfde als hierbove. De soort oorzaak van een bepaalde storing geven*
* **stm_prioriteit** : *De ernst van en bepaalde storing geven.* 

In [1]:
# libraries importeren 
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# dataset inlezen 
df = pd.read_csv('C:\Amaryllis_school_documents\Jaar_20_21\DDB\ddb_prorail\DDB_Prorail\data\sap_storing_data_hu_project.csv', low_memory=False)

### Data Understanding

In deze fase gaan we naar de inhoud kijken van de features variabelen en een betere inzicht te krijgen van onze data.

#### Data Collection 

Er wordt een data selectie gedaan. We gaan onze gekozen feature en target variabelen in onze dataset uithalen en in een niewe dataframe zetten. Verder zullen we naar de attributen van de  nieuwe data bekijken.

In [3]:
data = df[['stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_ddt']].dropna()
data['stm_aanntpl_ddt']  = data['stm_aanntpl_dd']  +" "+ data['stm_aanntpl_tijd']
data['stm_aanntpl_ddt']=pd.to_datetime(data['stm_aanntpl_ddt'], in)#https://stackoverflow.com/questions/37801321/python-pandas-convert-string-to-time-only
data['stm_fh_ddt']=pd.to_datetime(data['stm_fh_ddt'])
data['reparatietijd']   = data['stm_fh_ddt'] - data['stm_aanntpl_ddt']
#https://stackoverflow.com/questions/17951820/convert-hhmmss-to-minutes-using-python-pandas
data['reparatietijd']= data['reparatietijd'].dt.total_seconds().div(60)

SyntaxError: invalid syntax (<ipython-input-3-ec486aa2d3ea>, line 3)

In [ ]:
# Data selectie maken van de dataset.
df2  = df[['stm_oorz_groep', 'stm_oorz_code', 'stm_prioriteit']].copy()
df2['reparatietijd'] = data['reparatietijd']
df2.head()

#### Data Describe 

In dit gedeelte gaan we naar de eigenschappen van onze data bekijken.

In [ ]:
# Dimensie van de df2 bekijken
df2.shape

In [ ]:
# datatypes van de dataset bekijken 
df2.dtypes

**De meetniveaus van onze feature en target variabelen zijn de volgende:**
* **stm_oorz_groep** : Nominale waardes. Deze kolom moeten we *encoden* naar discrrete waardes
* **stm_oor_code**   : Nominale waardes. Deze kolom moeten we encoden naar discrrete waardes
* **stm_prioriteit** : Continu waardes.
* **reparatietijd**    : Continu waardes.

#### Data Explore

In [ ]:
# attributen van df2 bekijken
df2.describe()

In [ ]:
# Aantal ontbrekende waardes bekijken 
df2.isnull().sum()

Er is te zien dat we hebben te maken met veel NaN waardes in onze data. <br>
De aantal rijen die we zullen verloren, wordt berekend en weeregeven in percentage.Hierdoor wordt er bepaald of het het verstandig is om al deze NaN waardes te vewijderen.

In [ ]:
# bereken aantal rijen die wordt verloren bij Nan waardes
print('Rijen verloren door Nan oorzaakgroep : {}%'.format((df2['stm_oorz_groep'].isnull().sum()/845113)*100))
print('Rijen verloren door Nan oorzaakcode  : {}%'.format((df2['stm_oorz_groep'].isnull().sum()/845113)*100))
print('Rijen verloren door Nan oorzaakcode  : {}%'.format((df2['reparatietijd'].isnull().sum()/845113)*100))
#totaal
print('Totaal aantal rijen verloren : {} %'.format(df2.isnull().sum().sum()/845113*100))

We verliezen 38% van de data.

We gaan nu  kijken naar elke feature bekijken.

In [ ]:
df2['stm_oorz_groep'].unique()

In de kolom stm_orz_code gaan we de waarde met de csv bestand Oorzaakcodes verelijken om te zien of er wardes voorkomen die niet gelijk zijn.

In [ ]:
df2['stm_oorz_code'][df2['stm_oorz_code'].notnull()].sort_values().unique()

In [ ]:
oorzcodes = pd.read_csv('C:\Amaryllis_school_documents\Jaar_20_21\DDB\ddb_prorail\DDB_Prorail\data\Oorzaakcodes.csv', delimiter = ';')


We zien dat de waardes 33,  48,51 en 999.
Aan de hand van de bovenstaande code kunnnen we concluderen dat de we moeten df2 filteren die de rijen kiezen, waar de oorzaakcode tussen 130 en 299 zit. We zullen eerst zien hoeveel rijen bevatten deze deze oorzaak codes.

In [ ]:
print(len(df2[df2['stm_oorz_code'] == 33]))
print(len(df2[df2['stm_oorz_code'] == 48]))
print(len(df2[df2['stm_oorz_code'] == 51]))
print(len(df2[df2['stm_oorz_code'] == 999]))

Er zijn in totaal  105  rijen die deze waardes bevatten.We kunnen dus deze rijen uit onze data verwijderen.

In [ ]:
df2['stm_prioriteit'].unique()

Er is geen rij/ storing met een prioriteit van 3 en 6 of 7, maar we  kunnen ze misschien wel  voorspellen anagezien dat ze continu waardes zijn in onze Linear Regression model.

We gaan ook kijken in onze data of er dubbele rijen (storingen voorkomen) dmv duplicated()

In [ ]:
df2.columns

In [ ]:
df2['stm_oorz_groep'].unique()

In [ ]:
len(df2['stm_oorz_code'].dropna().unique())

In [ ]:
len(df2['stm_prioriteit'].unique())

### Data Preparation 

In deze fase gaan we de data opschonen.

In [ ]:
# Alle Nan waardes verwijderen
df2 = df2.dropna()
df2 = df2[(df2['stm_oorz_code'].isin([33, 48,51, 999]))== False] # de rijen verwijderen waar oorzaak groep is gelijk aan 33, 48, 51 en 999

We gaan de feature stm_oorz_groep en oorz_code encoden naar numerieke waardes met behulp van get_dummies().

In [ ]:
print(len(df2[df2.duplicated(keep='last')== False])) # kijk naar de dubbele rijen behalve de laatse occurrence. 
df2 = df2[df2.duplicated()== False]

In [ ]:
df2.isnull().sum()

In [ ]:
df2  = pd.get_dummies(df2, columns=["stm_oorz_groep"])

In [ ]:
df2  = pd.get_dummies(df2, columns=["stm_oorz_code"])

Als laatst gaan we een data selectie uitvoeren op de df2 waar de reparatietijd tussen 15 minuten en 8 uren zit.

Met behulp van de product owners, weten we dat we maximaal een storing met een reparatietijd van 8 uren moet voorspellen.

Wij kiezen voor een indergrens van 15 minuten. We gaan er  vanuit dat er geen drastische acties worden genomen als de oplossing van een storing korter dan 15minuten duurt.Voor deze storingen gaan wij er vanuit dat de dienstregeling niet aangepast hoeft te worden.

In [ ]:
# Data selectie is een range is van 15 en 480.
df2 = df2[(df2['reparatietijd'] >= 15) & (df2['reparatietijd'] <=  480)]

In [ ]:
percentile = np.percentile(df2['reparatietijd'].sort_values(), [25,50,75])
Q1 = percentile[0]
Q3 = percentile[2]
IQR =  Q3 - Q1

gns = Q1+1.5*IQR
print(gns)
print(len(df2[df2['reparatietijd']>gns])) 
df2['reparatietijd'].plot.box()

In [ ]:
#attributen bekijken
df2.describe()

In [ ]:
# Opgeschone data opslaan in een pickle bestand
data2.to_pickle("data.pkl")

## Nieuw Baseline Model 

Er zal een nieuwe baseline model opesteld worden op base van de opgeschone data en de data selectie die we hadden gedaan.

In [ ]:
#Nieuwe baseline model maken op bas evan de opgeschone data.
# kopie van de data voor zekerheid
df2_baseline = df2.copy()

In [ ]:
# Baselinne model    maken 
def baselinemodel(data,kolom):
    data['Median']=  data[kolom].median()
    # De kwaliteit/score van de model bepalen mbh van RMSE
    score =np.sqrt(mse(data.loc[:,kolom],data.loc[:,'Median']))
    return score

In [ ]:
basescore = baselinemodel(df2_baseline, 'reparatietijd')
basescore

In [ ]:
df2.to_pickle('df2.pkl')

## Linear regression model 

In [ ]:
def linearRegression(features, target):
    x_train, x_test, y_train, y_test = train_test_split(features, target) # splitsen in traiining en testing dataset
    # Geeft de dimensie van de opgesplitste datasets
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    linreg = LinearRegression()
    #fit model 
    linreg.fit(x_train, y_train)
    # Make prediction
    y_pred = linreg.predict(x_test)
    #Evaluate mbh van RMSE
    lrscore = np.sqrt(mse(y_test, y_pred))
    return lrscore

In [ ]:
#df2

In [ ]:
#Data opsplitsen in features en target variabelen.
features = df2.drop('reparatietijd', axis = 1)
target = df2['reparatietijd']

In [ ]:
lrscore = linearRegression(features, target)
lrscore

In [ ]:
# Bereken de afwijking met de baseline model  in percentage
((basescore - lrscore )/ basescore)*100 

##### Conclusie:
De score van onze Linear Regressie model is lager dan de score van de baseline model. <br>
Er is een verbetering van relatief 7,7 %. Dit is best een groot verschil , dus we kunnen zeggen dat deze model  een redelijke model om   toe te paseen.